In [1]:
pip install --upgrade fosforml

     |████████████████████████████████| 51kB 734kB/s eta 0:00:01
     |████████████████████████████████| 18.3MB 1.3MB/s eta 0:00:010
     |████████████████████████████████| 61kB 18.2MB/s eta 0:00:01
  Found existing installation: fosforml 1.1.7
    Uninstalling fosforml-1.1.7:
      Successfully uninstalled fosforml-1.1.7
  Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade numpy

  Using cached https://files.pythonhosted.org/packages/b9/14/78635daab4b07c0930c919d451b8bf8c164774e6a3413aed04a6d95758ce/numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
ERROR: snowflake-ml-python 1.5.0 has requirement numpy<2,>=1.23, but you'll have numpy 2.0.2 which is incompatible.
ERROR: scikit-learn 1.3.2 has requirement numpy<2.0,>=1.17.3, but you'll have numpy 2.0.2 which is incompatible.
ERROR: catboost 1.2.7 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 2.0.2 which is incompatible.
  Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [ ]:
pip install --upgrade seaborn

In [ ]:
!pip install pandas
!pip install snowflake-ml-python
!pip install requests

In [ ]:
pip install ydata-profiling --upgrade

In [ ]:
import pandas as pd
import numpy as np
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [ ]:
type(sf_df)

In [ ]:
df=sf_df.to_pandas()

In [ ]:
type(df)

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

## Converting to datetime data type

In [ ]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.corr(numeric_only=True)

In [ ]:
# Find unique dates
df['TRANS_DATE'].nunique()

In [ ]:
# Create features from date column
df['YEAR'] = df['TRANS_DATE'].dt.year
df['MONTH'] = df['TRANS_DATE'].dt.month
df['DAY'] = df['TRANS_DATE'].dt.day
df['DAY_OF_WEEK'] = df['TRANS_DATE'].dt.day_of_week + 1 # Monday is 1 and sunday is 7
df['QUARTER'] = df['TRANS_DATE'].dt.quarter
df['DAY_OF_YEAR'] = df['TRANS_DATE'].dt.dayofyear

In [ ]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

In [ ]:
df.head(5)

In [ ]:
'''from ydata_profiling import ProfileReport
profile=ProfileReport(df,explorative=True)
profile.to_file("autoeda_v1.html")'''

In [ ]:
df_dup = df[df.duplicated()].sort_values(by=['TRANS_DATE','SALES_UNITS','OUTLET_CODE'])

In [ ]:
df_unique = df.drop_duplicates()

In [ ]:
df_unique.shape

In [ ]:
df_check = df_unique.sort_values(by='TRANS_DATE')
df_check.head()

In [ ]:
df_all_dates = pd.date_range(start='2023-08-29', end='2024-08-27').tolist()

In [ ]:
missing_dates=set(df_all_dates) - set(df_unique['TRANS_DATE'])
len(missing_dates)

In [ ]:
# Calculate the week number for each date
df_check['WEEK_NUMBER'] = df_check['TRANS_DATE'].dt.isocalendar().week

In [ ]:
df_check.head()

In [ ]:
# Calculate the year difference from the first date
df_check['YEAR_DIFF'] = df_check['TRANS_DATE'].dt.year - df_check['TRANS_DATE'].dt.year.min()

# Calculate the continuous week number
df_check['CONTINUOUS_WEEK_NUMBER'] = df_check['WEEK_NUMBER'] + df_check['YEAR_DIFF'] * 52

# Adjust for the first year weeks
first_year_weeks = df_check[df_check['YEAR_DIFF'] == 0]['WEEK_NUMBER'].max()
df_check['CONTINUOUS_WEEK_NUMBER'] = df_check.apply(
    lambda row: row['CONTINUOUS_WEEK_NUMBER'] - (52 - first_year_weeks) if row['YEAR_DIFF'] > 0 else row['CONTINUOUS_WEEK_NUMBER'],
    axis=1
)
df_check['CONTINUOUS_WEEK_NUMBER']=df_check['CONTINUOUS_WEEK_NUMBER']-34

In [ ]:
import matplotlib.pyplot as plt

# Plot SALES_UNITS over CONTINUOUS_WEEK_NUMBER
plt.figure(figsize=(10, 5))
df_check.groupby('CONTINUOUS_WEEK_NUMBER')['SALES_UNITS'].sum().plot(kind='bar', color='orange')
plt.title('SALES_UNITS over CONTINUOUS_WEEK_NUMBER')
plt.xlabel('CONTINUOUS_WEEK_NUMBER')
plt.ylabel('SALES_UNITS')
plt.grid(True)
plt.show()

# Plot SALES_UNITS over DAY_OF_WEEK
plt.figure(figsize=(10, 5))
df_check.groupby('DAY_OF_WEEK')['SALES_UNITS'].sum().plot(kind='bar', color='salmon')
plt.title('SALES_UNITS over DAY_OF_WEEK')
plt.xlabel('Day of the Week')
plt.ylabel('SALES_UNITS')
plt.grid(True)
plt.show()

# Plot SALES_UNITS over TRANSACTION DATE
plt.figure(figsize=(10, 5))
df_check.groupby('TRANS_DATE')['SALES_UNITS'].sum().plot(kind='line', color='lightgreen')
plt.title('SALES_UNITS over DAY')
plt.xlabel('TRANSACTION DATE')
plt.ylabel('SALES_UNITS')
plt.grid(True)
plt.show()

import matplotlib.pyplot as plt
# Plot SALES_UNITS over MONTH
plt.figure(figsize=(10, 5))
df_check.groupby('MONTH')['SALES_UNITS'].sum().plot(kind='bar', color='skyblue')
plt.title('SALES_UNITS over MONTH')
plt.xlabel('Month')
plt.ylabel('SALES_UNITS')
plt.grid(True)
plt.show()

In [ ]:
df_final = df_check.drop(['WEEK_NUMBER','YEAR_DIFF'],axis=1)

In [ ]:
df_final.tail()

In [ ]:
df_final.info()

In [ ]:
df_final['OUTLET_CODE'].nunique()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Select features
features = ['SALES_VALUE', 'CITY', 'CATEGORY', 'SALES_UNITS', 'SALES_VOLUME']
df_selected = df_final[features]

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['SALES_VALUE', 'SALES_UNITS', 'SALES_VOLUME']),
        ('cat', OneHotEncoder(), ['CITY', 'CATEGORY'])
    ])

# Apply preprocessing
df_preprocessed = preprocessor.fit_transform(df_selected)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Calculate within-cluster sum of squares for different values of n_clusters
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(df_preprocessed)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method graph
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# Define the model
kmeans = KMeans(n_clusters=3, random_state=42)

# Fit the model
clusters = kmeans.fit_predict(df_preprocessed)

# Add cluster labels to the original DataFrame
df_final['CLUSTER'] = clusters

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Example visualization
sns.scatterplot(x='SALES_VALUE', y='SALES_UNITS', hue='CLUSTER', data=df_final)
plt.title('Clusters of OUTLET_CODE')
plt.show()

In [ ]:
df_final.head()

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le_product = LabelEncoder()
df_final['PRODUCT_CODE_ENCODED'] = le_product.fit_transform(df_final['PRODUCT_CODE'])

df_final = pd.get_dummies(df_final, columns=['DISTRIBUTOR_CODE', 'CATEGORY', 'SUBCATEGORY', 'BRAND', 'CITY','STATE', 'COUNTY'])

In [ ]:
df_final.head()

In [ ]:
df_featureset = df_final.copy()

In [ ]:
df_featureset.head()

In [ ]:
df_featureset.drop(['OC_CODE','OUTLET_CODE','PRODUCT_CODE','STREET'], axis=1, inplace=True)

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# Separate features and target
X = df_featureset.drop('SALES_UNITS', axis=1)
y = df_featureset['SALES_UNITS']